In [1]:
!pip install langchain_community
!pip install transformers
!pip install bitsandbytes
!pip install beautifulsoup4
!pip install langchain
!pip install langchain-community==0.2.1 langchain-core==0.2.1
!pip install sentence-transformers
!pip install faiss-gpu
!pip install accelerate
!pip install langchain-huggingface
!pip install huggingface-hub

In [11]:
import subprocess
import os
import wiki_scrape as ws
import RAG_chain as rc
import argparse
import transformers
from huggingface_hub import notebook_login

notebook_login()

In [12]:
retrieve_movies = False
retrieve_characters = False
retrieve_series = False
chunk_type = "Recursive"
chunk_size = 800
chunk_overlap = 100
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
#model_id = "Ba2han/Phi-3-Medium-Llamaish"

In [13]:
def chunk_docs(docs):
    headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2"), ("h3", "Header 3"), ("h4", "Header 4")]
    params = {}

    match chunk_type:
        case "Recursive":
            separators = ["\n\n", "\n", "(?<=\. )", " ", ""]
            params["separators"] = separators
            params["chunk_size"] = chunk_size
            params["chunk_overlap"] = chunk_overlap

            chunks = rc.chunk(docs, headers_to_split_on, chunk_type, **params)

    print("Number of chunks: ", len(chunks))
    print("Number of documents: ", len(docs))

    return chunks

Retrieve movies if needed

In [14]:
    if retrieve_movies:
        ws.export_movies()
    if retrieve_characters:
        ws.export_characters()
    if retrieve_series:
        ws.export_series()


Read the documents

In [15]:
print("Importing Documents")
docs = rc.read_docs("./web_pages")

Importing Documents


100%|██████████| 114/114 [00:00<00:00, 7078.37it/s]


Chunk documents

In [16]:
print("Creating chunks")
chunks = chunk_docs(docs)

Creating chunks
Number of chunks:  1296
Number of documents:  114


Create vector store

In [17]:
print("Creating a vector store")
embedding_model, vector_index = rc.create_vector_index_and_embedding_model(chunks)
print("Generating retriever")
retriever = vector_index.as_retriever()

Creating a vector store


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating retriever


Create the LLM pipeline

In [18]:
print("Downloading LLM model")
model, tokenizer = rc.generate_model(model_id)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
403 Client Error. (Request ID: Root=1-666334e7-111195237dc3fb54369fc2a9;70e6be34-aab0-4527-9b58-4f378b315497)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to ask for access.

In [ ]:
print("Creating the langchain pipeline")
llm_pipeline = rc.create_pipeline(model, tokenizer)

Create the chain

In [ ]:
print("Creating chain")
system_prompt = ("You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}")

chain = rc.create_qa_RAG_chain(llm_pipeline, retriever, system_prompt)

In [ ]:
chain.invoke({"input": "Who is Jyn Erso?"})